referensi 

https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_regression.html#sklearn.feature_selection.mutual_info_regression

In [1]:
# Import Library
import pandas as pd
import numpy as np
from sklearn.feature_selection import mutual_info_regression,SelectKBest 
from sklearn.model_selection import train_test_split, KFold, cross_validate
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time

In [2]:
#import dataset
df = pd.read_csv('/home/jupyter-17523142/Rama/Dataset/dataset_rama.csv')
X = df.drop(['No', 'Mango Cultivars', 'Vit C (mg/100g)', 'TA (mg/100g)', 'SSC (oBrix)', 'label'], axis = 1)
y_vitc = df.loc[:, 'Vit C (mg/100g)']

# separate dataset into train and test
x_train, x_test, y_train, y_test = train_test_split(
    X, y_vitc, test_size = 0.3, random_state=0)

# Seleksi Fitur Mutual Information

In [3]:
# Define MI feature selection
mi = SelectKBest(score_func=mutual_info_regression,k=100)

In [4]:
%%time

# Apply feature selection
mi.fit(x_train,y_train)

CPU times: user 1.83 s, sys: 30.1 ms, total: 1.86 s
Wall time: 1.86 s


SelectKBest(k=100,
            score_func=<function mutual_info_regression at 0x7fdd0289e840>)

In [5]:
# Create MI Value and features name
features_mi = pd.DataFrame(mi.scores_)
features = pd.DataFrame(x_train.columns)
features_mi = pd.concat([features,features_mi],axis=1)

# Assign the column name
features_mi.columns = ["Features","Mutual_Information_Value"]

# Print features score
print(features_mi.nlargest(100,columns="Mutual_Information_Value"))

     Features  Mutual_Information_Value
8        1003                  0.127114
24     1009.2                  0.125844
10     1003.8                  0.125737
23     1008.8                  0.124146
9      1003.4                  0.122458
...       ...                       ...
1004   1631.7                  0.100235
45     1017.6                  0.100201
207    1086.6                  0.100166
992    1619.4                  0.100157
296    1128.7                  0.100130

[100 rows x 2 columns]


In [6]:
# Fitur yang diseleksi
print(features_mi.nlargest(100,columns="Mutual_Information_Value").iloc[:,0].values.flatten().tolist())

['1003', '1009.2', '1003.8', '1008.8', '1003.4', '1007.7', '1008.1', '1004.2', '1009.6', '1002.6', '1010', '1008.5', '1002.2', '1001.8', '1133.7', '1012.8', '1007.3', '1138.2', '1005.7', '1134.2', '1011.2', '1006.5', '1006.1', '1004.5', '1001.1', '999.9', '1001.4', '1138.7', '1010.4', '1137.7', '1010.8', '1005.3', '1004.9', '1014', '1000.7', '1000.3', '1134.7', '1015.2', '1139.2', '1132.2', '1006.9', '1136.2', '1137.2', '1013.2', '1129.7', '1135.2', '1139.7', '1014.4', '1130.7', '1623.5', '1015.6', '1013.6', '1624.5', '2214.1', '1012', '1014.8', '1016', '1129.2', '2212.2', '1136.7', '1131.2', '1622.5', '1125.8', '1011.6', '1751.8', '1094', '1133.2', '1130.2', '1012.4', '1135.7', '1128.3', '1132.7', '1131.7', '1016.4', '1621.5', '1018', '1626.6', '1753', '1625.5', '1017.2', '2223.6', '1633.7', '1092.1', '2219.8', '1750.7', '2216', '2210.3', '1016.8', '2217.9', '2202.8', '1630.6', '1620.5', '2221.7', '1127.8', '1126.3', '1631.7', '1017.6', '1086.6', '1619.4', '1128.7']


In [7]:
cv = KFold(n_splits=10, random_state=1, shuffle=True)

# Linear Regression

In [8]:
%%time

#tentukan metode scoring yang digunakan
metrics = {'rmse': 'neg_root_mean_squared_error',
               'mse': 'neg_mean_squared_error',
               'mae': 'neg_mean_absolute_error',
               'r2': 'r2'} 

n_feat = range(1, 101)
min_mse = 1000
min_rmse = 1000
min_mae = 1000

for nfeat in n_feat:
    print("==================================================")
    start_time = time.time()

    #ambil n fitur input hasil seleksi fitur MI
    x_train_selected = x_train[features_mi.nlargest(100,columns="Mutual_Information_Value").iloc[0:nfeat, 0]]
    x_test_selected = x_test[features_mi.nlargest(100,columns="Mutual_Information_Value").iloc[0:nfeat, 0]]

    #Create a Linear Regression
    lr_model = LinearRegression()

    #Train the model using the training sets
    lr_model.fit(x_train_selected, y_train)
    y_pred_lr = lr_model.predict(x_test_selected)

    #hitung score model dari data train
    scores = cross_validate(lr_model, x_train_selected, y_train, scoring=metrics, cv=cv, return_train_score=True)

    print("MSE model Linear Regression data Train dengan " + str(nfeat) + " fitur: " 
          + str(abs(round(scores['train_mse'].mean(), 2))))
    print("RMSE model Linear Regression data Train dengan " + str(nfeat) + " fitur:"
          + str(abs(round(scores['train_rmse'].mean(), 2))))
    print("MAE model Linear Regression data Train dengan " + str(nfeat) + " fitur:"
          + str(abs(round(scores['train_mae'].mean(), 2))))
    print("R2 model Linear Regression data Train dengan " + str(nfeat) + " fitur:"
          + str(abs(round(scores['train_r2'].mean(), 2))))
    print("-------------------------")
    print("MSE model Linear Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_squared_error(y_test, y_pred_lr), 2))))
    print("RMSE model Linear Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_squared_error(y_test, y_pred_lr, squared = False), 2))))
    print("MAE model Linear Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_absolute_error(y_test, y_pred_lr), 2))))
    print("R2 model Linear Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(r2_score(y_test, y_pred_lr), 2))))
    print(" ")
    
    if(abs(round(mean_squared_error(y_test, y_pred_lr), 2) < min_mse)):
        min_mse = abs(round(mean_squared_error(y_test, y_pred_lr), 2))

    if(abs(round(mean_squared_error(y_test, y_pred_lr, squared = False), 2) < min_rmse)):
        min_rmse = abs(round(mean_squared_error(y_test, y_pred_lr, squared = False), 2))

    if(abs(round(mean_absolute_error(y_test, y_pred_lr), 2) < min_mae)):
        min_mae = abs(round(mean_absolute_error(y_test, y_pred_lr), 2))

    end_time = time.time()
    print("Total waktu: ", end_time - start_time)

print("====Nilai Performa Tertinggi====")
print("Nilai MSE model terendah: " + str(min_mse))
print("Nilai RMSE model terendah: " + str(min_rmse))
print("Nilai MAE model terendah: " + str(min_mae))
print("Nilai R2 model tertinggi: " + str(max_r2))

MSE model Linear Regression data Train dengan 1 fitur: 188.47
RMSE model Linear Regression data Train dengan 1 fitur:13.73
MAE model Linear Regression data Train dengan 1 fitur:11.06
R2 model Linear Regression data Train dengan 1 fitur:0.02
-------------------------
MSE model Linear Regression data Test dengan 1 fitur:183.45
RMSE model Linear Regression data Test dengan 1 fitur:13.54
MAE model Linear Regression data Test dengan 1 fitur:10.41
R2 model Linear Regression data Test dengan 1 fitur:0.06
 
Total waktu:  0.1784043312072754
MSE model Linear Regression data Train dengan 2 fitur: 188.24
RMSE model Linear Regression data Train dengan 2 fitur:13.72
MAE model Linear Regression data Train dengan 2 fitur:11.09
R2 model Linear Regression data Train dengan 2 fitur:0.02
-------------------------
MSE model Linear Regression data Test dengan 2 fitur:184.0
RMSE model Linear Regression data Test dengan 2 fitur:13.56
MAE model Linear Regression data Test dengan 2 fitur:10.43
R2 model Linear R

# Random Forest Regression (n = 100)

In [9]:
%%time

#tentukan metode scoring yang digunakan
metrics = {'rmse': 'neg_root_mean_squared_error',
               'mse': 'neg_mean_squared_error',
               'mae': 'neg_mean_absolute_error',
               'r2': 'r2'} 

n_feat = range(1, 101)
n_trees = [100]
min_mse = 1000
min_rmse = 1000
min_mae = 1000

for nfeat in n_feat:
    for ntrees in n_trees:
        print("==================================================")
        start_time = time.time()

        #ambil n fitur input hasil seleksi fitur MI
        x_train_selected = x_train[features_mi.nlargest(100,columns="Mutual_Information_Value").iloc[0:nfeat, 0]]
        x_test_selected = x_test[features_mi.nlargest(100,columns="Mutual_Information_Value").iloc[0:nfeat, 0]]

        #Create a Random Forest Regression
        rfg_model = RandomForestRegressor(n_estimators=ntrees)
        
        #Train the model using the training sets
        rfg_model.fit(x_train_selected, y_train)
        y_pred_rfg=rfg_model.predict(x_test_selected)

        #hitung score model dari data train
        scores = cross_validate(rfg_model, x_train_selected, y_train, scoring=metrics, cv=cv, return_train_score=True)

        print("MSE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:"
              + str(abs(round(scores['train_mse'].mean(), 2))))
        print("RMSE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:"
              + str(abs(round(scores['train_rmse'].mean(), 2))))
        print("MAE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:"
              + str(abs(round(scores['train_mae'].mean(), 2))))
        print("R2 model Random Forest Regression data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:"
              + str(abs(round(scores['train_r2'].mean(), 2))))
        print("-------------------------")
        print("MSE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(abs(round(mean_squared_error(y_test, y_pred_rfg), 2))))
        print("RMSE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(abs(round(mean_squared_error(y_test, y_pred_rfg, squared = False), 2))))
        print("MAE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(abs(round(mean_absolute_error(y_test, y_pred_rfg), 2))))
        print("R2 model Random Forest Regression data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(abs(round(r2_score(y_test, y_pred_rfg), 2))))
        print(" ")

        if(abs(round(mean_squared_error(y_test, y_pred_rfg), 2) < min_mse)):
            min_mse = abs(round(mean_squared_error(y_test, y_pred_rfg), 2))

        if(abs(round(mean_squared_error(y_test, y_pred_rfg, squared = False), 2) < min_rmse)):
            min_rmse = abs(round(mean_squared_error(y_test, y_pred_rfg, squared = False), 2))

        if(abs(round(mean_absolute_error(y_test, y_pred_rfg), 2) < min_mae)):
            min_mae = abs(round(mean_absolute_error(y_test, y_pred_rfg), 2))
            
        end_time = time.time()
        print("Total waktu: ", end_time - start_time)

print("====Nilai Performa Tertinggi====")
print("Nilai MSE model terendah: " + str(min_mse))
print("Nilai RMSE model terendah: " + str(min_rmse))
print("Nilai MAE model terendah: " + str(min_mae))
print("Nilai R2 model tertinggi: " + str(max_r2))

MSE model Random Forest Regression data Train dengan 1 fitur dan 100 trees:42.66
RMSE model Random Forest Regression data Train dengan 1 fitur dan 100 trees:6.53
MAE model Random Forest Regression data Train dengan 1 fitur dan 100 trees:5.03
R2 model Random Forest Regression data Train dengan 1 fitur dan 100 trees:0.78
-------------------------
MSE model Random Forest Regression data Test dengan 1 fitur dan 100 trees:253.2
RMSE model Random Forest Regression data Test dengan 1 fitur dan 100 trees:15.91
MAE model Random Forest Regression data Test dengan 1 fitur dan 100 trees:12.62
R2 model Random Forest Regression data Test dengan 1 fitur dan 100 trees:0.47
 
Total waktu:  1.2134017944335938
MSE model Random Forest Regression data Train dengan 2 fitur dan 100 trees:43.17
RMSE model Random Forest Regression data Train dengan 2 fitur dan 100 trees:6.57
MAE model Random Forest Regression data Train dengan 2 fitur dan 100 trees:5.15
R2 model Random Forest Regression data Train dengan 2 fit

# Random Forest Regression (n = 150)

In [10]:
%%time

#tentukan metode scoring yang digunakan
metrics = {'rmse': 'neg_root_mean_squared_error',
               'mse': 'neg_mean_squared_error',
               'mae': 'neg_mean_absolute_error',
               'r2': 'r2'} 

n_feat = range(1, 101)
n_trees = [150]
min_mse = 1000
min_rmse = 1000
min_mae = 1000

for nfeat in n_feat:
    for ntrees in n_trees:
        print("==================================================")
        start_time = time.time()

        #ambil n fitur input hasil seleksi fitur MI
        x_train_selected = x_train[features_mi.nlargest(100,columns="Mutual_Information_Value").iloc[0:nfeat, 0]]
        x_test_selected = x_test[features_mi.nlargest(100,columns="Mutual_Information_Value").iloc[0:nfeat, 0]]

        #Create a Gaussian Classifier
        rfg_model = RandomForestRegressor(n_estimators=ntrees)
        
        #Train the model using the training sets
        rfg_model.fit(x_train_selected, y_train)
        y_pred_rfg=rfg_model.predict(x_test_selected)

        #hitung score model dari data train
        scores = cross_validate(rfg_model, x_train_selected, y_train, scoring=metrics, cv=cv, return_train_score=True)

        print("MSE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:"
              + str(abs(round(scores['train_mse'].mean(), 2))))
        print("RMSE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:"
              + str(abs(round(scores['train_rmse'].mean(), 2))))
        print("MAE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:"
              + str(abs(round(scores['train_mae'].mean(), 2))))
        print("R2 model Random Forest Regression data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:"
              + str(abs(round(scores['train_r2'].mean(), 2))))
        print("-------------------------")
        print("MSE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(abs(round(mean_squared_error(y_test, y_pred_rfg), 2))))
        print("RMSE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(abs(round(mean_squared_error(y_test, y_pred_rfg, squared = False), 2))))
        print("MAE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(abs(round(mean_absolute_error(y_test, y_pred_rfg), 2))))
        print("R2 model Random Forest Regression data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(abs(round(r2_score(y_test, y_pred_rfg), 2))))
        print(" ")

        if(abs(round(mean_squared_error(y_test, y_pred_rfg), 2) < min_mse)):
            min_mse = abs(round(mean_squared_error(y_test, y_pred_rfg), 2))

        if(abs(round(mean_squared_error(y_test, y_pred_rfg, squared = False), 2) < min_rmse)):
            min_rmse = abs(round(mean_squared_error(y_test, y_pred_rfg, squared = False), 2))

        if(abs(round(mean_absolute_error(y_test, y_pred_rfg), 2) < min_mae)):
            min_mae = abs(round(mean_absolute_error(y_test, y_pred_rfg), 2))

        if(abs(round(r2_score(y_test, y_pred_rfg), 2) < max_r2)):
            max_r2 = abs(round(r2_score(y_test, y_pred_rfg), 2))

        end_time = time.time()
        print("Total waktu: ", end_time - start_time)

print("====Nilai Performa Tertinggi====")
print("Nilai MSE model terendah: " + str(min_mse))
print("Nilai RMSE model terendah: " + str(min_rmse))
print("Nilai MAE model terendah: " + str(min_mae))
print("Nilai R2 model tertinggi: " + str(max_r2))

MSE model Random Forest Regression data Train dengan 1 fitur dan 150 trees:42.85
RMSE model Random Forest Regression data Train dengan 1 fitur dan 150 trees:6.54
MAE model Random Forest Regression data Train dengan 1 fitur dan 150 trees:5.05
R2 model Random Forest Regression data Train dengan 1 fitur dan 150 trees:0.78
-------------------------
MSE model Random Forest Regression data Test dengan 1 fitur dan 150 trees:257.16
RMSE model Random Forest Regression data Test dengan 1 fitur dan 150 trees:16.04
MAE model Random Forest Regression data Test dengan 1 fitur dan 150 trees:12.73
R2 model Random Forest Regression data Test dengan 1 fitur dan 150 trees:0.49
 
Total waktu:  1.4737350940704346
MSE model Random Forest Regression data Train dengan 2 fitur dan 150 trees:43.03
RMSE model Random Forest Regression data Train dengan 2 fitur dan 150 trees:6.55
MAE model Random Forest Regression data Train dengan 2 fitur dan 150 trees:5.12
R2 model Random Forest Regression data Train dengan 2 fi

# Random Forest Regression n = 200

In [11]:
%%time

#tentukan metode scoring yang digunakan
metrics = {'rmse': 'neg_root_mean_squared_error',
               'mse': 'neg_mean_squared_error',
               'mae': 'neg_mean_absolute_error',
               'r2': 'r2'} 

#tentukan total fitur yang digunakan dalam proses klasifikasi SVM ini
n_feat = range(1, 101)
n_trees = [200]
min_mse = 1000
min_rmse = 1000
min_mae = 1000
max_r2 = 0

for nfeat in n_feat:
    for ntrees in n_trees:
        print("==================================================")
        start_time = time.time()

        #ambil n fitur input hasil seleksi fitur MI
        x_train_selected = x_train[features_mi.nlargest(100,columns="Mutual_Information_Value").iloc[0:nfeat, 0]]
        x_test_selected = x_test[features_mi.nlargest(100,columns="Mutual_Information_Value").iloc[0:nfeat, 0]]

        #Create a Gaussian Classifier
        rfg_model = RandomForestRegressor(n_estimators=ntrees)
        
        #Train the model using the training sets
        rfg_model.fit(x_train_selected, y_train)
        y_pred_rfg=rfg_model.predict(x_test_selected)

        #hitung score model dari data train
        scores = cross_validate(rfg_model, x_train_selected, y_train, scoring=metrics, cv=cv, return_train_score=True)

        print("MSE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:"
              + str(abs(round(scores['train_mse'].mean(), 2))))
        print("RMSE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:"
              + str(abs(round(scores['train_rmse'].mean(), 2))))
        print("MAE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:"
              + str(abs(round(scores['train_mae'].mean(), 2))))
        print("R2 model Random Forest Regression data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:"
              + str(abs(round(scores['train_r2'].mean(), 2))))
        print("-------------------------")
        print("MSE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(abs(round(mean_squared_error(y_test, y_pred_rfg), 2))))
        print("RMSE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(abs(round(mean_squared_error(y_test, y_pred_rfg, squared = False), 2))))
        print("MAE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(abs(round(mean_absolute_error(y_test, y_pred_rfg), 2))))
        print("R2 model Random Forest Regression data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(abs(round(r2_score(y_test, y_pred_rfg), 2))))
        print(" ")

        if(abs(round(mean_squared_error(y_test, y_pred_rfg), 2) < min_mse)):
            min_mse = abs(round(mean_squared_error(y_test, y_pred_rfg), 2))

        if(abs(round(mean_squared_error(y_test, y_pred_rfg, squared = False), 2) < min_rmse)):
            min_rmse = abs(round(mean_squared_error(y_test, y_pred_rfg, squared = False), 2))

        if(abs(round(mean_absolute_error(y_test, y_pred_rfg), 2) < min_mae)):
            min_mae = abs(round(mean_absolute_error(y_test, y_pred_rfg), 2))

        if(abs(round(r2_score(y_test, y_pred_rfg), 2) < max_r2)):
            max_r2 = abs(round(r2_score(y_test, y_pred_rfg), 2))

        end_time = time.time()
        print("Total waktu: ", end_time - start_time)

print("====Nilai Performa Tertinggi====")
print("Nilai MSE model terendah: " + str(min_mse))
print("Nilai RMSE model terendah: " + str(min_rmse))
print("Nilai MAE model terendah: " + str(min_mae))
print("Nilai R2 model tertinggi: " + str(max_r2))

MSE model Random Forest Regression data Train dengan 1 fitur dan 200 trees:42.57
RMSE model Random Forest Regression data Train dengan 1 fitur dan 200 trees:6.52
MAE model Random Forest Regression data Train dengan 1 fitur dan 200 trees:5.03
R2 model Random Forest Regression data Train dengan 1 fitur dan 200 trees:0.78
-------------------------
MSE model Random Forest Regression data Test dengan 1 fitur dan 200 trees:256.51
RMSE model Random Forest Regression data Test dengan 1 fitur dan 200 trees:16.02
MAE model Random Forest Regression data Test dengan 1 fitur dan 200 trees:12.72
R2 model Random Forest Regression data Test dengan 1 fitur dan 200 trees:0.49
 
Total waktu:  1.9610717296600342
MSE model Random Forest Regression data Train dengan 2 fitur dan 200 trees:43.04
RMSE model Random Forest Regression data Train dengan 2 fitur dan 200 trees:6.55
MAE model Random Forest Regression data Train dengan 2 fitur dan 200 trees:5.12
R2 model Random Forest Regression data Train dengan 2 fi